In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
import os 
import tarfile
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

if os.path.exists("/content/drive/MyDrive/Colab Notebooks/"):
    COLAB_PATH = "/content/drive/MyDrive/Colab Notebooks/大伯 - ML_Code"
else:
    COLAB_PATH = "/content/drive/MyDrive/UTKFace-Code/"
os.chdir(COLAB_PATH)

!pwd
!ls -l

/content/drive/MyDrive/UTKFace-Code
total 306378
-rw------- 1 root root  38599552 Oct 29 08:22 Best_model_params.h5
-rw------- 1 root root 167472288 Nov  7 02:03 best_xception_race_weights.h5
drwx------ 2 root root      4096 Oct 21 12:21 UTKFace
-rw------- 1 root root     34195 Dec  8 17:53 UTKFace-CNN-race-classifier.ipynb
drwx------ 2 root root      4096 Oct 21 10:42 UTKFace-data-cleaning
drwx------ 2 root root      4096 Oct 22 06:23 UTKFace_data_cleaning_TFRecords
drwx------ 2 root root      4096 Oct 12 00:37 UTKFace_not_split_TFRecords
-r-------- 1 root root 106634631 Mar 23  2017 UTKFace.tar.gz
-rw------- 1 root root    972660 Dec  8 10:33 UTKFace-train_test_split-to-tfrecords.ipynb


In [ ]:
# Compress {train, test dataset} tfrecords.gz to {train, test} tfrecords.tar
# train_path = "./UTKFace_TFRecords/train/"
# test_path = "./UTKFace_TFRecords/test/"

# def build_tarfile_to_compress_tfrecords(name, paths):
#     for i in range(len(name)):
#         with tarfile.open(f"./UTKFace_TFRecords/{name[i]}/UTKFace_{name[i]}_dataset.tfrecords.tar", "w") as tar:
#             path_list = lambda x: os.listdir(x)
#             for j in path_list(paths[i]):
#                 tar.add(paths[i]+j)

# build_tarfile_to_compress_tfrecords(("train", "test"), (train_path, test_path))

In [ ]:
# Decompress to retrieve train, test dataset tfrecords.gz
# tar_paths = "./UTKFace_TFRecords/"

# def decompress_tarfile_to_retrieve_tfrecords_gz(name, paths):
#     for i in range(len(name)):
#         with tarfile.open(paths+name[i]+f"/UTKFace_{name[i]}_dataset.tfrecords.tar") as tar:
#             tar.extractall()

# decompress_tarfile_to_retrieve_tfrecords_gz(("train", "test"), tar_paths)

In [ ]:
# Parse(Preprocess) TFRecord 

def parse_tfrecord(tfrecord):
    feature = {
        # "shape": tf.io.FixedLenFeature([3], tf.int64),
        "image": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.int64)
    }
    parse_example = tf.io.parse_single_example(tfrecord, feature)
    image = tf.io.parse_tensor(parse_example["image"], out_type=tf.uint8)
    image = tf.reshape(image, shape=[200, 200, 3])

    return image, parse_example["label"]

In [ ]:
# Preprocess with data augmentation, tf.data pipeline and decompress tfrecords

from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, callbacks, Model, regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input, Xception

AUTOTUNE = tf.data.AUTOTUNE

# Data Augmentation use keras ImageDataGenerator
# opt1:
# data_generator = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True
# )
    
# data_generator.flow(image, label, batch_size = 32)

# opt2:
data_augmentation = models.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1, 0.1)
])


# Transfer learning
def image_preprocess(image, label):
    resized_img = tf.image.resize(image, [224, 224])
    input_img_resized = preprocess_input(resized_img)

    return input_img_resized, label


def preprocess_tfrecord(paths, n_threads = AUTOTUNE, compress_attr = "GZIP"):
    path_list = tf.data.Dataset.list_files(paths, seed = 42)

    # decompress tfrecords
    dataset = tf.data.TFRecordDataset(path_list, compression_type = compress_attr, num_parallel_reads = n_threads)
    dataset = dataset.map(parse_tfrecord, num_parallel_calls = n_threads)

    return dataset


def OneHotEncode(image, label):
    one_hot_label = tf.one_hot(label, len(race_classes))

    return image, one_hot_label


def preprocess_and_quick_read_dataset(dataset, n_threads = AUTOTUNE, buf_size = None, batch_size = 32, augment = False):
    # Zero-Centered - scale: [0~255]->[0~1] (* Don't use Zero-Centered if Transfer Learning)
    # dataset = dataset.map(lambda x, y: ((tf.cast(x, dtype = tf.float32) / 255.), y), num_parallel_calls = n_threads)
    dataset = dataset.map(lambda x, y: (tf.cast(x, dtype = tf.float32), y), num_parallel_calls = n_threads)
    
    # Transfer Learning Architecture required img shape
    dataset = dataset.map(image_preprocess, num_parallel_calls = n_threads)

    # One Hot Encode
    dataset = dataset.map(OneHotEncode, num_parallel_calls = n_threads)

    dataset = dataset.cache()

    # Use Data Augment only on training set
    if augment:
        dataset = dataset.map(lambda x, y: (data_augmentation(x, training = True), y), num_parallel_calls = n_threads)

    if buf_size:
        dataset = dataset.shuffle(buf_size)

    return dataset.batch(batch_size).prefetch(n_threads)

In [ ]:
# Plot UTKFace race img

data_clean = False
if data_clean:
    tfrecords_path = "UTKFace_data_cleaning_TFRecords/"
else:
    tfrecords_path = "UTKFace_not_split_TFRecords/"

train_paths = os.path.join(tfrecords_path, "train/UTKFace_train_*.tfrecords.gz")
valid_paths = os.path.join(tfrecords_path, "valid/UTKFace_valid_*.tfrecords.gz")
test_paths = os.path.join(tfrecords_path, "test/UTKFace_test_*.tfrecords.gz")


# Race: White, Black, Asia, Indian, Others(Hispanic, Latino, Middle Eastern)
race_classes = ["White", "Black", "Asia", "Indian", "Others"]


# UTKFace dataset => train_set: 80%, valid_set: 10%, test_set: 10%  
# Batch_size = 32

train_set = preprocess_and_quick_read_dataset(preprocess_tfrecord(train_paths), buf_size = 20000 if data_clean else 24000, augment = True)
valid_set = preprocess_and_quick_read_dataset(preprocess_tfrecord(valid_paths))
test_set = preprocess_and_quick_read_dataset(preprocess_tfrecord(test_paths))

In [ ]:
# Check the Y label dimensions
# for X, Y in train_set.take(1):
#     print(Y.numpy().shape)

In [ ]:
# Plot train_set take(1)

# plt.figure(figsize = (6, 6), tight_layout = True)
# for X, Y in train_set.take(1):
#     for i in range(6):
#         plt.subplot(3, 3, i+1)
#         plt.axis("off")
#         plt.imshow(X[i].numpy(), cmap = "binary")
#         # plt.title(f"{race_classes[int(Y[i].numpy())]}: ({str(Y[i].numpy())})", fontsize = 14)
        
#         # OneHot transform
#         plt.title(f"{race_classes[np.argmax(Y[i])]}: ({np.argmax(Y[i])})", fontsize = 14)

In [ ]:
# Plot test_set take(1)

# plt.figure(figsize = (6, 6), tight_layout = True)
# for X, Y in test_set.take(1):
#     for i in range(6):
#         plt.subplot(3, 3, i+1)
#         plt.axis("off")
#         plt.imshow(X[i].numpy(), cmap = "binary")
#         # plt.title(f"{race_classes[int(Y[i].numpy())]}: ({str(Y[i].numpy())})", fontsize = 14)

#         # OneHot transform
#         plt.title(f"{race_classes[np.argmax(Y[i])]}: ({np.argmax(Y[i])})", fontsize = 14)

In [ ]:
# Golab GPU devices status

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
!nvidia-smi -L

Num GPUs Available:  0
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
# Use TPU

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_host(resolver.master())
tf.tpu.experimental.initialize_tpu_system(resolver)
tpu_strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.25.174.130:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.25.174.130:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
# # CNN Model - VGG16

# def vgg16_model(decay = 1e-4):
#     model = models.Sequential()
    
#     # block 1
#     model.add(layers.Conv2D(64, (3, 3), padding = "same", activation = "relu", kernel_initializer = "he_normal", kernel_regularizer = regularizers.L2(decay)
#                             , input_shape = (200, 200, 3)))
#     model.add(layers.Conv2D(64, (3, 3), padding = "same", activation = "relu", kernel_regularizer = regularizers.L2(decay)))
#     model.add(layers.BatchNormalization())
#     model.add(layers.MaxPooling2D((2, 2), strides = 2))
    
#     # block 2
#     model.add(layers.Conv2D(128, (3, 3), padding = "same", activation = "relu", kernel_initializer = "he_normal", kernel_regularizer = regularizers.L2(decay)))
#     model.add(layers.Conv2D(128, (3, 3), padding = "same", activation = "relu", kernel_regularizer = regularizers.L2(decay)))
#     model.add(layers.BatchNormalization())
#     model.add(layers.MaxPooling2D((2, 2), strides = 2))

#     # block 3
#     model.add(layers.Conv2D(256, (3, 3), padding = "same", activation = "relu", kernel_initializer = "he_normal", kernel_regularizer = regularizers.L2(decay)))
#     model.add(layers.Conv2D(256, (3, 3), padding = "same", activation = "relu", kernel_regularizer = regularizers.L2(decay)))
#     model.add(layers.Conv2D(256, (3, 3), padding = "same", activation = "relu", kernel_regularizer = regularizers.L2(decay)))
#     model.add(layers.BatchNormalization())
#     model.add(layers.MaxPooling2D((2, 2), strides = 2))

#     # block 3
#     model.add(layers.Conv2D(512, (3, 3), padding = "same", activation = "relu", kernel_initializer = "he_normal", kernel_regularizer = regularizers.L2(decay)))
#     model.add(layers.Conv2D(512, (3, 3), padding = "same", activation = "relu", kernel_regularizer = regularizers.L2(decay)))
#     model.add(layers.Conv2D(512, (3, 3), padding = "same", activation = "relu", kernel_regularizer = regularizers.L2(decay)))
#     model.add(layers.BatchNormalization())
#     model.add(layers.MaxPooling2D((2, 2), strides = 2))

#     # block 4
#     model.add(layers.Conv2D(512, (3, 3), padding = "same", activation = "relu", kernel_initializer = "he_normal", kernel_regularizer = regularizers.L2(decay)))
#     model.add(layers.Conv2D(512, (3, 3), padding = "same", activation = "relu", kernel_regularizer = regularizers.L2(decay)))
#     model.add(layers.Conv2D(512, (3, 3), padding = "same", activation = "relu", kernel_regularizer = regularizers.L2(decay)))
#     model.add(layers.BatchNormalization())
#     model.add(layers.MaxPooling2D((2, 2), strides = 2))

#     # flatten
#     model.add(layers.Flatten())
#     model.add(layers.Dense(1024, activation = "relu", kernel_regularizer = regularizers.L2(decay)))
#     model.add(layers.Dropout(0.5))
#     # model.add(layers.Dense(4096, activation = "relu", kernel_regularizer = regularizers.L2(decay)))
#     # model.add(layers.Dropout(0.5))
#     model.add(layers.Dense(512, activation = "relu", kernel_regularizer = regularizers.L2(decay)))
#     model.add(layers.Dropout(0.5))
#     model.add(layers.Dense(len(race_classes), activation = "softmax"))

#     # compile the model
#     opt = tf.keras.optimizers.Adam(learning_rate = 0.001, clipvalue = 0.1)

#     model.compile(loss = "categorical_crossentropy", optimizer = opt,
#                   metrics = ["accuracy"])  # OneHotEncode transform -> loss = "categorical_crossentropy" 
    
#     return model

In [ ]:
# vgg16 = vgg16_model()
# vgg16.summary()

In [ ]:
# # CNN Model

# def cnn_model(lr = 1e-4, epochs = None):

#     model = models.Sequential()
    
#     # step 1:
#     model.add(layers.Conv2D(16, (3,3), padding = "same", activation = "relu", input_shape = (200, 200, 3)))
#     model.add(layers.BatchNormalization())
#     model.add(layers.MaxPooling2D((3,3)))
#     model.add(layers.Dropout(0.25))

#     # step 2:
#     model.add(layers.Conv2D(32, (3,3), padding = "same", activation = "relu"))
#     model.add(layers.BatchNormalization())
#     model.add(layers.MaxPooling2D((2,2)))
#     model.add(layers.Dropout(0.25))

#     # step 3:
#     model.add(layers.Conv2D(32, (3,3), padding = "same", activation = "relu"))
#     model.add(layers.BatchNormalization())
#     model.add(layers.MaxPooling2D((2,2)))
#     model.add(layers.Dropout(0.25))

#     # step 4:
#     model.add(layers.Conv2D(64, (3,3), padding = "same", activation = "relu"))
#     model.add(layers.BatchNormalization())
#     model.add(layers.MaxPooling2D((2,2)))
#     model.add(layers.Dropout(0.25))

#     # flatten
#     model.add(layers.Flatten())
#     model.add(layers.Dense(128, activation = "relu"))
#     model.add(layers.BatchNormalization())
#     model.add(layers.Dropout(0.5))
#     model.add(layers.Dense(len(race_classes), activation = "softmax"))

#     # compile the model
#     opt = tf.keras.optimizers.Adam(learning_rate = lr, clipvalue = 0.1, decay = lr / epochs)
#     model.compile(loss = "categorical_crossentropy", optimizer = opt,
#                 metrics = ["accuracy"])  # OneHotEncode transform -> loss = "categorical_crossentropy"

#     return model

In [ ]:
# Epochs = 100

# cnn = cnn_model(epochs = Epochs)
# cnn.summary()

In [ ]:
# Fit model

# Initialize parameters
# lr = 1e-4
# Batch = 32
# Epochs = 100

# # the loss remains the same as {Patience} times, early stop will start.
# Patience = 15   

# cnn = cnn_model(epochs = Epochs)
# # cnn.summary()

# # cnn: best_race_weights.h5, vgg16: best_vgg16_race_weights.h5
# race_classifier_weights = os.path.join(COLAB_PATH, "best_vgg16_race_weights.h5")

# # Learning scheduling
# # performance_scheduler = callbacks.ReduceLROnPlateau(factor = 0.2, patience = 5, min_lr = 1e-6)

# check_point = callbacks.ModelCheckpoint(race_classifier_weights, monitor = "val_loss", save_best_only = True)
# early_stop = callbacks.EarlyStopping(monitor = "val_loss", patience = Patience, restore_best_weights = True)

# call_backs = [check_point, early_stop]  # performance_scheduler

# # model: vgg16, cnn -> 73 epochs, val_acc: ≈77% 
# history = cnn.fit(train_set,
#                   epochs = Epochs,
#                 #    steps_per_epoch = 477,
#                   validation_data = valid_set,
#                 #   initial_epoch = 73,
#                 #    validation_steps = 60,
#                   callbacks = call_backs)

In [ ]:
# test_loss, test_acc = cnn.evaluate(test_set, verbose = 2)

# print(f"test loss: {test_loss:.4f}")
# print(f"test acc: {test_acc:.4f}")

In [ ]:
# Transfer learning - GPU

xception_model = Xception(weights = "imagenet",
                          include_top = False)
gbl_avgpool = layers.GlobalAveragePooling2D()(xception_model.output)
output = layers.Dense(len(race_classes), activation = "softmax")(gbl_avgpool)
model = Model(inputs = xception_model.input, outputs = output)


# Xception layers include_top = False, not have last two layers(GlobalAveragePooling2D(), Dense())
# for index, layer in enumerate(xception_model.layers):
#     print(index, layer.name)

model.summary()

In [ ]:
# Transfer learning - TPU strategy

# def create_model():
#     xception_model = Xception(weights = "imagenet",
#                               include_top = False)
#     gbl_avgpool = layers.GlobalAveragePooling2D()(xception_model.output)
#     output = layers.Dense(len(race_classes), activation = "softmax")(gbl_avgpool)
#     model = Model(inputs = xception_model.input, outputs = output)

#     # Freeze xception_model layers weights
#     for layer in xception_model.layers:
#         layer.trainable = False
    
#     opt = tf.keras.optimizers.SGD(learning_rate = 0.15, momentum = 0.9, decay = 0.01)    # sgd: 0.03 <= lr <= 0.3
#     model.compile(loss = "categorical_crossentropy", optimizer = opt, 
#                   metrics = ["accuracy"])
    
#     return model

# with tpu_strategy.scope():
#     model = create_model()

In [ ]:
# Freeze xception_model layers weights
for layer in xception_model.layers:
    layer.trainable = False

# initialize parameters
Epochs = 15


# Compile and Fit
# SGD:
# --- lr: 0.15 -> epoch: 11, val_acc: 0.6791(loss = 0.9065), epoch: 15, val_acc: 0.6728(loss = 0.8944) ----> quick convergence !
# --- lr: 0.2 -> epoch: 12, val_acc: 0.6749(loss = 0.9021) ----> quick convergence !
# --- lr: 0.3 -> epoch: 12, val_acc: 0.6628(loss = 0.9258)
# --- lr: 0.03 -> epoch: 9, val_acc: 0.6560
# --- lr: 0.05 -> epoch: 13, val_acc: 0.6529, epoch: 20, val_acc: 0.6628(loss = 0.9678)
# opt = tf.keras.optimizers.SGD(learning_rate = 0.15, momentum = 0.9, decay = 0.01)    # sgd: 0.03 <= lr <= 0.3

# Adam:
opt = tf.keras.optimizers.Adam(learning_rate = 0.001, decay = 0.01)  # Adam not good !


model.compile(loss = "categorical_crossentropy", optimizer = opt, 
              metrics = ["accuracy"])

history = model.fit(train_set, 
                    epochs = Epochs,
                    # steps_per_epoch = int(0.1*594),  
                    validation_data = valid_set,
                    # validation_steps = int(0.1*75)   
                   )

InvalidArgumentError: ignored

In [ ]:
# EPOCHS = 12

# start_lr = 0.00001
# min_lr = 0.00001
# max_lr = 0.00005  # * tpu_strategy.num_replicas_in_sync
# rampup_epochs = 5
# sustain_epochs = 0
# exp_decay = .8

# def lrfn(epoch):
#   if epoch < rampup_epochs:
#     return (max_lr - start_lr)/rampup_epochs * epoch + start_lr
#   elif epoch < rampup_epochs + sustain_epochs:
#     return max_lr
#   else:
#     return (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr
    
# lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda epoch: lrfn(epoch), verbose=True)

# rang = np.arange(EPOCHS)
# y = [lrfn(x) for x in rang]
# plt.plot(rang, y)
# print('Learning rate per epoch:')

In [ ]:
# Unfreeze xception_model layers weights
for layer in xception_model.layers:
    layer.trainable = True

race_classifier_weights = os.path.join(COLAB_PATH, "best_xception_race_weights.h5")

# initialize parameters
# Epochs = 12
Patience = 10


# Compile and Fit -> learning slow
opt = tf.keras.optimizers.SGD(learning_rate = 0.01, momentum = 0.9, 
                              nesterov = True, decay = 0.001)  
model.compile(loss = "categorical_crossentropy", optimizer = opt,
              metrics = ["accuracy"])

# Learning scheduling
# performance_scheduler = callbacks.ReduceLROnPlateau(factor = 0.5, patience = 2, min_lr = 1e-6)

check_point = callbacks.ModelCheckpoint(race_classifier_weights, monitor = "val_loss", save_best_only = True)
early_stop = callbacks.EarlyStopping(monitor = "val_loss", patience = Patience, restore_best_weights = True)

call_backs = [check_point, early_stop]  # performance_scheduler

history = model.fit(train_set,
                     epochs = Epochs,
                    #  steps_per_epoch = int(0.75*477),   
                     validation_data = valid_set,
                    #  validation_steps = int(0.15*75),  
                     callbacks = call_backs
                   )  

Epoch 1/12
477/477 [==============================] - 819s 1s/step - loss: 0.7748 - accuracy: 0.7240 - val_loss: 0.7875 - val_accuracy: 0.7184


/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/12
477/477 [==============================] - 810s 1s/step - loss: 0.5427 - accuracy: 0.8106 - val_loss: 0.5038 - val_accuracy: 0.8222
Epoch 3/12
477/477 [==============================] - 807s 1s/step - loss: 0.4631 - accuracy: 0.8403 - val_loss: 0.5098 - val_accuracy: 0.8154
Epoch 4/12
477/477 [==============================] - 806s 1s/step - loss: 0.4035 - accuracy: 0.8635 - val_loss: 0.4698 - val_accuracy: 0.8285
Epoch 5/12
477/477 [==============================] - 808s 1s/step - loss: 0.3537 - accuracy: 0.8764 - val_loss: 0.5017 - val_accuracy: 0.8228
Epoch 6/12
477/477 [==============================] - 808s 1s/step - loss: 0.3148 - accuracy: 0.8898 - val_loss: 0.4967 - val_accuracy: 0.8217
Epoch 7/12
477/477 [==============================] - 807s 1s/step - loss: 0.2678 - accuracy: 0.9079 - val_loss: 0.5098 - val_accuracy: 0.8306
Epoch 8/12
477/477 [==============================] - 807s 1s/step - loss: 0.2405 - accuracy: 0.9170 - val_loss: 0.5212 - val_accuracy: 0.8338

Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7f2ea79bbf80>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


Epoch 9/12


KeyboardInterrupt: ignored

In [ ]:
test_loss, test_acc = model.evaluate(test_set, verbose = 2)

print(f"test loss: {test_loss:.4f}")
print(f"test acc: {test_acc:.4f}")

60/60 - 20s - loss: 0.6102 - accuracy: 0.8248
test loss: 0.6102
test acc: 0.8248
